**Topic 2 – End-to-end Machine Learning project**

*Welcome to ECIC Housing Development Ltd! Your task is to predict median house values in Californian districts, given a number of features from these districts.*

This is a very long notebook.  It will take many minutes to complete executing the codes in this entire notebook.  For fun, the textbok called the company Machine Learning Housing Corp.  I call it ECIC Housing Development Ltd.  It is not a real company.


# Setup
Remember to import all the libraries/modules/classes used in your Python prgram before using them.  The import does not need to be in the beginning, but has to be imported before using.

First, let's import a few common modules, ensure MatplotLib plots figures inline and prepare a function to save the figures.

In [ ]:
# Python ≥3.5 and Scikit-Learn ≥0.20 are required
import sys
import sklearn
import numpy as np
import os

# To plot pretty figures (for report etc)
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "end_to_end_project"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Get the Data

## Download the Data

In [ ]:
# It is a good practice to "package" your tasks in functions!
import os   # does not matter if you import twice (just in case!)
import tarfile  # for de-compress tarfiles
import urllib.request

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    if not os.path.isdir(housing_path):
        os.makedirs(housing_path)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()

In [ ]:
fetch_housing_data()

In [ ]:
# We shall be using pandas DataFrame to manage all our data
# Again, use function for clear documentation and modularization
import pandas as pd

def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

## Take a Quick Look at the Data Structure

In [ ]:
# Useful to get a "peek" at the data using .head()
housing = load_housing_data()
housing.head()

In [ ]:
# Useful to know what you have in the DataFrame
# The structure is MORE important than the actual data!!
housing.info()

In [ ]:
# As this column is non-numerical, useful to see what it is!
housing["ocean_proximity"].value_counts()

In [ ]:
# For numeric data - get an overall statisitics!
housing.describe()

In [ ]:
# Get a histograph plot for numeric data to have a big picture of your data
%matplotlib inline
import matplotlib.pyplot as plt
housing.hist(bins=50, figsize=(20,15))
save_fig("attribute_histogram_plots")
plt.show()

## Create a Test Set

In [ ]:
# We do random sampling, hence it is important to make the out
# (repeatibility) same every time we run but give a fix seed (42)
np.random.seed(42)

In [ ]:
import numpy as np

# For illustration only. Sklearn has train_test_split()
# Again, package the task in a function!!!!!!!!
def split_train_test(data, test_ratio):
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(len(data) * test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return data.iloc[train_indices], data.iloc[test_indices]

In [ ]:
# Get the training set and testing set with one call!
# Print the "info" of each dataset
train_set, test_set = split_train_test(housing, 0.2)
print("Training - ")
train_set.info()
print("Testing - ")
test_set.info()

In [ ]:
# create a random ID for each instance for easier management later
from zlib import crc32

def test_set_check(identifier, test_ratio):
    return crc32(np.int64(identifier)) & 0xffffffff < test_ratio * 2**32

def split_train_test_by_id(data, test_ratio, id_column):
    ids = data[id_column]
    in_test_set = ids.apply(lambda id_: test_set_check(id_, test_ratio))
    return data.loc[~in_test_set], data.loc[in_test_set]

Cycle Redundancy Check is a fine but may not work if you want to add additional data later.  You would have to redo everything from the beginning.  A better alternative is to use a Hash function.
The following code is easier than what is used in the textbook, by using the md5 hash function from hashlib.

In [ ]:
# using Hash Function is an even better option for sampling
import hashlib

def test_set_check(identifier, test_ratio, hash=hashlib.md5):
    return hash(np.int64(identifier)).digest()[-1] < 256 * test_ratio

In [ ]:
# we need to add the `index` column to our datasets
housing_with_id = housing.reset_index()   # adds an `index` column
train_set, test_set = split_train_test_by_id(housing_with_id, 0.2, "index")

In [ ]:
# have a look at the test_set (can do the same for train_set)
test_set.info()

In [ ]:
# yet another alternative of ID is using 'longitude' and 'latitude'
housing_with_id["id"] = housing["longitude"] * 1000 + housing["latitude"]
train_set, test_set = split_train_test_by_id(housing_with_id, 0.2, "id")

In [ ]:
# look at the first instances of test_set using .head()
test_set.head()

In [ ]:
# all the above are for explanation - use sklearn's train_test_split!!!!
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)

In [ ]:
test_set.head()

In [ ]:
# experts tell you 'median_income' is most important feature, have a look
housing["median_income"].hist()

In [ ]:
# you want to see if you need to stratify your data according to `median_income`
housing["income_cat"] = pd.cut(housing["median_income"],
                               bins=[0., 1.5, 3.0, 4.5, 6., np.inf],
                               labels=[1, 2, 3, 4, 5])

In [ ]:
# look at each bin
housing["income_cat"].value_counts()

In [ ]:
housing["income_cat"].hist()

In [ ]:
# yes, you need to stratify, use StratifiedShuffleSplit class in sklearn
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

In [ ]:
# note that the index for stratification is an numpy ndarray!
type(train_index)

In [ ]:
# check if the stratification agrees with the distribution!!
strat_test_set["income_cat"].value_counts() / len(strat_test_set)

In [ ]:
# Great!!  They agree!
housing["income_cat"].value_counts() / len(housing)

In [ ]:
# this is nothing to do with our model, but to see the effect of stratification!
def income_cat_proportions(data):
    return data["income_cat"].value_counts() / len(data)

train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)

compare_props = pd.DataFrame({
    "Overall": income_cat_proportions(housing),
    "Stratified": income_cat_proportions(strat_test_set),
    "Random": income_cat_proportions(test_set),
}).sort_index()
compare_props["Rand. %error"] = 100 * compare_props["Random"] / compare_props["Overall"] - 100
compare_props["Strat. %error"] = 100 * compare_props["Stratified"] / compare_props["Overall"] - 100

In [ ]:
compare_props

In [ ]:
# the 'income_cat' column is used only for stratification (in sampling)
# now that we have the stratified training and testing set, drop that column!
for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis=1, inplace=True)

# Discover and Visualize the Data to Gain Insights

In [ ]:
# make a copy of the stratified training dataset in 'housing' (our training DataFrame)
# all the work so far is to get the right set of data!!
housing = strat_train_set.copy()

## Visualizing Geographical Data

In [ ]:
# essential to get a picture of the data we are dealing with - visualization!
housing.plot(kind="scatter", x="longitude", y="latitude")
save_fig("bad_visualization_plot")

In [ ]:
# demonstrate use of 'alpha' (transparency) in plots
housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.1)
save_fig("better_visualization_plot")

The argument `sharex=False` fixes a display bug (the x-axis values and legend were not displayed). This is a temporary fix (see: https://github.com/pandas-dev/pandas/issues/10611 ). Thanks to Wilmer Arellano for pointing it out.

In [ ]:
# use size of circles ('s') for population and colour ('c') for median house value
housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.4,
             s=housing["population"]/100, label="population", figsize=(10,7),
             c="median_house_value", cmap=plt.get_cmap("jet"), colorbar=True,
             sharex=False)
plt.legend()
save_fig("housing_prices_scatterplot")

In [ ]:
# Download the California image
# this is not in the book or the powerpoint notes - extravagent!!
images_path = os.path.join(PROJECT_ROOT_DIR, "images", "end_to_end_project")
os.makedirs(images_path, exist_ok=True)
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
filename = "california.png"
print("Downloading", filename)
url = DOWNLOAD_ROOT + "images/end_to_end_project/" + filename
urllib.request.urlretrieve(url, os.path.join(images_path, filename))

In [ ]:
import matplotlib.image as mpimg
california_img=mpimg.imread(os.path.join(images_path, filename))
ax = housing.plot(kind="scatter", x="longitude", y="latitude", figsize=(10,7),
                  s=housing['population']/100, label="Population",
                  c="median_house_value", cmap=plt.get_cmap("jet"),
                  colorbar=False, alpha=0.4)
plt.imshow(california_img, extent=[-124.55, -113.80, 32.45, 42.05], alpha=0.5,
           cmap=plt.get_cmap("jet"))
plt.ylabel("Latitude", fontsize=14)
plt.xlabel("Longitude", fontsize=14)

prices = housing["median_house_value"]
tick_values = np.linspace(prices.min(), prices.max(), 11)
cbar = plt.colorbar(ticks=tick_values/prices.max())
cbar.ax.set_yticklabels(["$%dk"%(round(v/1000)) for v in tick_values], fontsize=14)
cbar.set_label('Median House Value', fontsize=16)

plt.legend(fontsize=16)
save_fig("california_housing_prices_plot")
plt.show()

## Looking for Correlations

In [ ]:
# get the correlaton matrix - this is how to see which attributes are more useful
corr_matrix = housing.corr()

In [ ]:
corr_matrix["median_house_value"].sort_values(ascending=False)

In [ ]:
# from pandas.tools.plotting import scatter_matrix # For older versions of Pandas
from pandas.plotting import scatter_matrix

attributes = ["median_house_value", "median_income", "total_rooms",
              "housing_median_age"]
scatter_matrix(housing[attributes], figsize=(12, 8))
save_fig("scatter_matrix_plot")

In [ ]:
# 'median_income' by far is most correlated and useful, have a closer look!
housing.plot(kind="scatter", x="median_income", y="median_house_value",
             alpha=0.1)
plt.axis([0, 16, 0, 550000])
save_fig("income_vs_house_value_scatterplot")

## Experimenting with Attribute Combinations

In [ ]:
# Experimenting our attributes is important to get max benefit from data.
housing["rooms_per_household"] = housing["total_rooms"]/housing["households"]
housing["bedrooms_per_room"] = housing["total_bedrooms"]/housing["total_rooms"]
housing["population_per_household"]=housing["population"]/housing["households"]

In [ ]:
# experiment shows `bedrooms_per_room` is significant (negatively correlated!)
corr_matrix = housing.corr()
corr_matrix["median_house_value"].sort_values(ascending=False)

In [ ]:
# 'room_per_household' is also useful (but not as significant as 'bedrooms_per_room')
housing.plot(kind="scatter", x="rooms_per_household", y="median_house_value",
             alpha=0.2)
plt.axis([0, 5, 0, 520000])
plt.show()

In [ ]:
# check out the statistics of `housing` DataFrame
housing.describe()

# Prepare the Data for Machine Learning Algorithms

In [ ]:
# drop the labels from the training dataset and set up a separate `housing_labels1 dataset
housing = strat_train_set.drop("median_house_value", axis=1) # drop labels for training set
housing_labels = strat_train_set["median_house_value"].copy()

## Data Cleaning

In the book 3 options for data cleaning are listed:

```python
housing.dropna(subset=["total_bedrooms"])    # option 1
housing.drop("total_bedrooms", axis=1)       # option 2
median = housing["total_bedrooms"].median()  # option 3
housing["total_bedrooms"].fillna(median, inplace=True)
```

To demonstrate each of them, let's create a copy of the housing dataset, but keeping only the rows that contain at least one null. Then it will be easier to visualize exactly what each option does:

In [ ]:
sample_incomplete_rows = housing[housing.isnull().any(axis=1)].head()
sample_incomplete_rows

In [ ]:
sample_incomplete_rows.dropna(subset=["total_bedrooms"])    # option 1

In [ ]:
sample_incomplete_rows.drop("total_bedrooms", axis=1)       # option 2

In [ ]:
median = housing["total_bedrooms"].median()
sample_incomplete_rows["total_bedrooms"].fillna(median, inplace=True) # option 3

In [ ]:
sample_incomplete_rows

In [ ]:
# at the end, let us use the 'SimpleImputer' of sklearn and "median" strategy to clean data
# the above are to explain what could be done using available packages, not by a single call!
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy="median")

Remove the text attribute because median can only be calculated on numerical attributes:

In [ ]:
# but wait... median can only be applied to numerical attributes, hence need to do more...
housing_num = housing.drop("ocean_proximity", axis=1)
# alternatively: housing_num = housing.select_dtypes(include=[np.number])

In [ ]:
# now apply the .fit() method to housing_num (numerical subset of housing)
# you will learn about .fit(), .transform() and .fit_transform() in lecture
# these concepts are important!
imputer.fit(housing_num)
# the result of .fit() method is to get the statistics from the dataset (fitting the data!)

In [ ]:
# the results are store in the variable 'statistics_'
# lst us have a look at it
imputer.statistics_

Check that this is the same as manually computing the median of each attribute:

In [ ]:
# the same as computing the median fore each attribute
housing_num.median().values

Transform the training set:

In [ ]:
# Now!!  Ready to transform the housing_num data (using imputer) and put the transformed data in X (captial)
X = imputer.transform(housing_num)

In [ ]:
# now, get the DataFrame from X, and setup the `housing_tr` - the transformed housing dataset
# the follwing code cells are just to examine some of the transformed data in `housing_tr`
housing_tr = pd.DataFrame(X, columns=housing_num.columns,
                          index=housing.index)

In [ ]:
housing_tr.loc[sample_incomplete_rows.index.values]

In [ ]:
imputer.strategy

In [ ]:
housing_tr = pd.DataFrame(X, columns=housing_num.columns,
                          index=housing_num.index)

In [ ]:
housing_tr.head()

## Handling Text and Categorical Attributes

After preprocessing numerical data using Imputer and set up our transformers, now let's preprocess the **categorical input feature**, `*ocean_proximity*`:

In [ ]:
# have a quick look at `ocean_proximity` atrribute in the first 10 instances 
# 'housing_cat' is similar to 'housing_num" but for categorical data
housing_cat = housing[["ocean_proximity"]]
housing_cat.head(10)

In [ ]:
# need to encode description 'strings' into a datastructure that is meaningful to computer
# use 'OrdinalEncoder' class in 'preprocessing' module of sklearn
# note that .fit_transform does 2 things together - fitting, then transform
from sklearn.preprocessing import OrdinalEncoder

ordinal_encoder = OrdinalEncoder()
housing_cat_encoded = ordinal_encoder.fit_transform(housing_cat)
housing_cat_encoded[:10]  # you can see that the encoded is an numpy array

In [ ]:
# orindal transform assumes certain ordinal order, but this is not our case
# '<1H OCEAN' is not better or worst than 'INLAND', look at the categories
ordinal_encoder.categories_

In [ ]:
# use OneHotEncoder in sklearn solves the problem!
# it generate a sparse matrix, each row has a '1' in the category it belongs to
# and '0' elsewhere, hence sparse matrix is more efficient to store the information
from sklearn.preprocessing import OneHotEncoder

cat_encoder = OneHotEncoder()
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)
housing_cat_1hot

By default, the `OneHotEncoder` class returns a sparse array, but we can convert it to a dense array if needed by calling the `toarray()` method:

In [ ]:
# we can convert the sparse matrix back to a 'ordinary' dense matrix
housing_cat_1hot.toarray()

Alternatively, you can set `sparse=False` when creating the `OneHotEncoder`:

In [ ]:
# setting the 'sparse' parameter to 'False' is easier!
cat_encoder = OneHotEncoder(sparse=False)
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)
housing_cat_1hot

In [ ]:
cat_encoder.categories_

## Custom Transformers

Let's create a custom transformer to add extra attributes.  This is way more complex and you may not fully understand what it does.  It does not matter if you skip this for now.  Basically, if you cannot find the necessary transformers or algorithms in sklearn, you can write your own customized transformers yourself and this example shows you how this is done.

We indeed use this custom transform to add new attributes to our Dataframe before we explore the various ML models.  The relevant attributes in the original dataset are with indexes (column) [3, 4, 5, 6] for [rooms, bedrooms, population, housefholds].

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

# column index
rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room=True): # no *args or **kargs
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self  # nothing else to do
    def transform(self, X):
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
        population_per_household = X[:, population_ix] / X[:, households_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household,
                         bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)
housing_extra_attribs = attr_adder.transform(housing.values)

Note that I hard coded the indices (3, 4, 5, 6) for concision and clarity in the book, but it would be much cleaner to get them dynamically, like this.  'cleaner' here mean that you do not hard program it (then anything you change in future you need to remember to change the hard-code).

In [ ]:
# cleaner approach to replace hard-coding
col_names = "total_rooms", "total_bedrooms", "population", "households"
rooms_ix, bedrooms_ix, population_ix, households_ix = [
    housing.columns.get_loc(c) for c in col_names] # get the column indices

Also, `housing_extra_attribs` is a NumPy array, we've lost the column names (unfortunately, that's a problem with Scikit-Learn). To recover a `DataFrame`, you could run this:

In [ ]:
# putting back in the attribute names (remember, NumPy arrays have to be the same data type, hence no names!)
# Pandas DataFrame has atribute names!
housing_extra_attribs = pd.DataFrame(
    housing_extra_attribs,
    columns=list(housing.columns)+["rooms_per_household", "population_per_household"],
    index=housing.index)
housing_extra_attribs.head()

## Transformation Pipelines

Now let's build a pipeline for preprocessing the numerical attributes:

In [ ]:
# Transformation pipelines make it clean and also clear for reusability
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('attribs_adder', CombinedAttributesAdder()),
        ('std_scaler', StandardScaler()),
    ])

housing_num_tr = num_pipeline.fit_transform(housing_num)

In [ ]:
# have a look at the transformed numerical dataset in `housing_num_tr` DataFrame
# pandas is clever in showing you a few rows and columns at the 'head' and 'tail' of the DataFrame
# the number of rows and columns to show by default can also be changed (using appropriate parameters!)
housing_num_tr

In [ ]:
# use sklearn's 'ColumnTransform' to construct the full_pipeline
# the final result in executing the full pipeline is now in housing_prepared
# all the work so far is TO PREPARE the data!!!!!!
from sklearn.compose import ColumnTransformer

num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", OneHotEncoder(), cat_attribs),
    ])

housing_prepared = full_pipeline.fit_transform(housing)

In [ ]:
# looking at the data is ... not too useful!
housing_prepared

In [ ]:
# but look at it shape (and even .info() is probably more useful
housing_prepared.shape

# Select and Train a Model

## Training and Evaluating on the Training Set

In [ ]:
# we are now ready to try our well-prepared datasets (in Pandas DataFrame) on various model
# try the simplest - linear regression model
# remembers from Topic 1, .fit(X, y)?  The same here, so simply!
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, housing_labels)

# when you run the above, nothing appears to happen!  In fact, it has finished fitting the model
# and all relevant coeficients that define the model are store within the model!!

In [ ]:
# let's try the full preprocessing pipeline on a few training instances
# this is to gain some insight as to how the model is working
some_data = housing.iloc[:5]
some_labels = housing_labels.iloc[:5]
# you need to transform the subset from training data (5 instances from training dataset)
# you can see why the transformation pipeline is so important and useful
some_data_prepared = full_pipeline.transform(some_data)

print("Predictions:", lin_reg.predict(some_data_prepared))

Compare against the actual values:

In [ ]:
# as we have the labels for the training data, we can compare
print("Labels:", list(some_labels))

In [ ]:
# have a look at the sample data we are dealing with
some_data_prepared

In [ ]:
# we can use the `mean_squared_error` function in sklearn ('metrics' module)
# then sqrt method in numpy to get RMSE
from sklearn.metrics import mean_squared_error

housing_predictions = lin_reg.predict(housing_prepared)
lin_mse = mean_squared_error(housing_labels, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

**Note**: since Scikit-Learn 0.22, you can get the RMSE directly by calling the `mean_squared_error()` function with `squared=False`.

In [ ]:
# alternatively, we can computer the mean_absolute_error
from sklearn.metrics import mean_absolute_error

lin_mae = mean_absolute_error(housing_labels, housing_predictions)
lin_mae

In [ ]:
# let us try another model, how about decision tree
# this is to demonstrate how easy it is to try different models
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(random_state=42)
tree_reg.fit(housing_prepared, housing_labels)

In [ ]:
# ZERO ERROR, something must be wrong (characteristic of this model!)
housing_predictions = tree_reg.predict(housing_prepared)
tree_mse = mean_squared_error(housing_labels, housing_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

## Better Evaluation Using Cross-Validation

In [ ]:
# using training data this way to validate the tune models in not good
# this is because the model was trained WITH the same data
# better use cross-validation (using decision tree model still)
from sklearn.model_selection import cross_val_score

scores = cross_val_score(tree_reg, housing_prepared, housing_labels,
                         scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)

In [ ]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

display_scores(tree_rmse_scores)

In [ ]:
# can try to get the cross validation score using linear regression model 
lin_scores = cross_val_score(lin_reg, housing_prepared, housing_labels,
                             scoring="neg_mean_squared_error", cv=10)
lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

**Note**: we specify `n_estimators=100` is the default value is going to change to 100 in Scikit-Learn now (not shown in book).

In [ ]:
# try one more model - random forest regressor, but no cross validation
# remember to set 'random_state' to 42, to ensure reproducibiiity
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(n_estimators=100, random_state=42)
forest_reg.fit(housing_prepared, housing_labels)

In [ ]:
housing_predictions = forest_reg.predict(housing_prepared)
forest_mse = mean_squared_error(housing_labels, housing_predictions)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

In [ ]:
# try the same using cross_val_score
from sklearn.model_selection import cross_val_score

forest_scores = cross_val_score(forest_reg, housing_prepared, housing_labels,
                                scoring="neg_mean_squared_error", cv=10)
forest_rmse_scores = np.sqrt(-forest_scores)
display_scores(forest_rmse_scores)

In [ ]:
# now we have a few rounds for, say, Linear Regression cross_val_scores, let us see their statistics
scores = cross_val_score(lin_reg, housing_prepared, housing_labels, scoring="neg_mean_squared_error", cv=10)
pd.Series(np.sqrt(-scores)).describe()

In [ ]:
# let us try Support Vector Machine (SVM)
from sklearn.svm import SVR

svm_reg = SVR(kernel="linear")
svm_reg.fit(housing_prepared, housing_labels)
housing_predictions = svm_reg.predict(housing_prepared)
svm_mse = mean_squared_error(housing_labels, housing_predictions)
svm_rmse = np.sqrt(svm_mse)
svm_rmse

# Fine-Tune Your Model

## Grid Search
When the search space is large (with various models and their respective ***hyperparameters*** (make sure you understand what are hyperparamters), we need to automatic the entire search process. The solution is **grid search**.

In [ ]:
# Example of Grid Search using RandomForestRegressor
from sklearn.model_selection import GridSearchCV

param_grid = [
    # try 12 (3×4) combinations of hyperparameters
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    # then try 6 (2×3) combinations with bootstrap set as False
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
    ]

forest_reg = RandomForestRegressor(random_state=42)
# train across 5 folds, that's a total of (12+6)*5=90 rounds of training 
grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)
grid_search.fit(housing_prepared, housing_labels)

The best hyperparameter combination found:

In [ ]:
# .best_params_ variable stores the best choise of hyperparameters
grid_search.best_params_

In [ ]:
# .best_estimator_ reports also the model used (with the respective hyperparameters)
grid_search.best_estimator_

Let's look at the score of each hyperparameter combination tested during the grid search:

In [ ]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

In [ ]:
pd.DataFrame(grid_search.cv_results_)

## Randomized Search
Instead of the data scientist set the hyperparameters in grid search, how about randomly try various values for hyperparameters.  This is called ***Randomized Search***.

In [ ]:

# Example of Randomized Search with Cross-Validation
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_distribs = {
        'n_estimators': randint(low=1, high=200),
        'max_features': randint(low=1, high=8),
    }

forest_reg = RandomForestRegressor(random_state=42)
rnd_search = RandomizedSearchCV(forest_reg, param_distributions=param_distribs,
                                n_iter=10, cv=5, scoring='neg_mean_squared_error', random_state=42)
rnd_search.fit(housing_prepared, housing_labels)

In [ ]:
# Print the cross-validation results for various hyperparameters in each iteration
# random search is controlled by the number of iteration 'n_iter', which was set to 10
cvres = rnd_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

## Analyze the Best Models and Their Errors

In [ ]:
# you can also analyse the feature importance of the best model
feature_importances = grid_search.best_estimator_.feature_importances_
feature_importances

In [ ]:
extra_attribs = ["rooms_per_hhold", "pop_per_hhold", "bedrooms_per_room"]
#cat_encoder = cat_pipeline.named_steps["cat_encoder"] # old solution
cat_encoder = full_pipeline.named_transformers_["cat"]
cat_one_hot_attribs = list(cat_encoder.categories_[0])
attributes = num_attribs + extra_attribs + cat_one_hot_attribs
sorted(zip(feature_importances, attributes), reverse=True)

## Evaluate Your System on the Test Set

In [ ]:
# we now have the best model and the best hyperparameters according to cross-validation
# the remaining job is to try with the test data (that the model has never seen)
final_model = grid_search.best_estimator_

# use stratified test data, remove the target column and copy this in a label array
X_test = strat_test_set.drop("median_house_value", axis=1)
y_test = strat_test_set["median_house_value"].copy()

# use the tranform pipeline to transform the test dataset
# the model is already trained earlier, and all coeficients have been "learned"
# now predict!!!
X_test_prepared = full_pipeline.transform(X_test)
final_predictions = final_model.predict(X_test_prepared)

final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)

In [ ]:
# The final result after ALL THESE WORK
final_rmse

We can compute a 95% confidence interval for the test RMSE:

In [ ]:
# Finally, computer the 95% condience level
# with 95% chance that the RMSE of the predicted house value is withih [45893, 49774]
# this can be compared with past experience from experts (without ML), and inform investment by the Company
# job well done!!!!
from scipy import stats

confidence = 0.95
squared_errors = (final_predictions - y_test) ** 2
np.sqrt(stats.t.interval(confidence, len(squared_errors) - 1,
                         loc=squared_errors.mean(),
                         scale=stats.sem(squared_errors)))

We could compute the interval manually like this:

In [ ]:
m = len(squared_errors)
mean = squared_errors.mean()
tscore = stats.t.ppf((1 + confidence) / 2, df=m - 1)
tmargin = tscore * squared_errors.std(ddof=1) / np.sqrt(m)
np.sqrt(mean - tmargin), np.sqrt(mean + tmargin)

Alternatively, we could use a z-scores rather than t-scores:

In [ ]:
zscore = stats.norm.ppf((1 + confidence) / 2)
zmargin = zscore * squared_errors.std(ddof=1) / np.sqrt(m)
np.sqrt(mean - zmargin), np.sqrt(mean + zmargin)

# Extra material
The following are extra material not coverd in lecture or the book.

## A full pipeline with both preparation and prediction

In [ ]:
full_pipeline_with_predictor = Pipeline([
        ("preparation", full_pipeline),
        ("linear", LinearRegression())
    ])

full_pipeline_with_predictor.fit(housing, housing_labels)
full_pipeline_with_predictor.predict(some_data)

## Model persistence using joblib

In [ ]:
my_model = full_pipeline_with_predictor

In [ ]:
import joblib
joblib.dump(my_model, "my_model.pkl") # DIFF
#...
my_model_loaded = joblib.load("my_model.pkl") # DIFF

## Example SciPy distributions for `RandomizedSearchCV`

In [ ]:
from scipy.stats import geom, expon
geom_distrib=geom(0.5).rvs(10000, random_state=42)
expon_distrib=expon(scale=1).rvs(10000, random_state=42)
plt.hist(geom_distrib, bins=50)
plt.show()
plt.hist(expon_distrib, bins=50)
plt.show()